In [38]:
import os 
path= os.getcwd()

if path.endswith('notebooks'):
    os.chdir('../')

In [50]:
# import dependencies
import numpy as np 
import pandas as pd 
import seaborn as sns 
from pathlib import Path
import matplotlib.pyplot as plt
from src.Home_Premium_Prediction.utils import create_directories, read_yaml
from src.Home_Premium_Prediction.constants import CONFIG_FILE_PATH

class DataTransfromationConfig:
    def __init__(self, data_transformation_dir: Path, train_data_path: Path, processed_data_path: Path):
        self.data_transformation_dir = data_transformation_dir
        self.train_data_path = train_data_path
        self.processed_data_path = processed_data_path

class DataTransformationConfigManager:
    def __init__(self, config_file=CONFIG_FILE_PATH):
        self.config_file = read_yaml(config_file)
    
    def get_data_transformation_config(self) -> DataTransfromationConfig:
        create_directories([self.config_file['data_transformation']['data_transformation_dir']])

        return DataTransfromationConfig(
            data_transformation_dir=self.config_file['data_transformation']['data_transformation_dir'],
            train_data_path=self.config_file['data_transformation']['train_data_path'],
            processed_data_path=self.config_file['data_transformation']['processed_data_path']
        )

class DataTransformation:
    def __init__(self, config: DataTransfromationConfig):
        self.config = config
    
    def read_data(self):
        self.df = pd.read_csv(self.config.train_data_path)

    def drop_unwanted_cols(self):
        self.df = self.df.drop(columns=['uuid', 'quote_id'], axis=1)

    def splitting_data_into_features_and_target(self):
        self.features = self.df.drop(columns=['Premium'], axis=1)
        self.target = self.df['Premium']

    def prepare_data_for_catboost(self):
        self.categorical_columns = self.features.select_dtypes(include=['category', 'object']).columns.to_list()
        return self.features, self.target, self.categorical_columns

# ✅ Main runner block
if __name__ == "__main__":
    config = DataTransformationConfigManager().get_data_transformation_config()
    transformer = DataTransformation(config)
    
    transformer.read_data()
    transformer.drop_unwanted_cols()
    transformer.splitting_data_into_features_and_target()
    
    X, y, cat_cols = transformer.prepare_data_for_catboost()
    
    print("✅ Features and target prepared for CatBoost")
    print("Categorical columns:", cat_cols)


created directory at: artifacts/data_transformation
✅ Features and target prepared for CatBoost
Categorical columns: ['property_type', 'coverage_level', 'broker_name', 'ownership_status', 'energy_efficiency_rating', 'pcd']


In [1]:
X # done

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Bilal Ahmad\Desktop\Home-Insurance-Price-Predictive-Model\.venv\Lib\site-packages\IPython\core\interactiveshell.py", line 3670, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
    ~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Bilal Ahmad\AppData\Local\Temp\ipykernel_3472\2411327631.py", line 1, in <module>
    X # done
    ^
NameError: name 'X' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Bilal Ahmad\Desktop\Home-Insurance-Price-Predictive-Model\.venv\Lib\site-packages\IPython\core\interactiveshell.py", line 2176, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
        etype, value, tb, tb_offset=tb_offset
    )
  File "c:\Users\Bilal Ahmad\Desktop\Home-Insurance-Price-Predictive-Model\.venv\Lib\site-packages\IPython\core\ultratb.py", line 1182, in structured_traceback
    return Format

In [49]:
y

0         1740.49
1          677.71
2         1441.61
3         1957.38
4         1543.64
           ...   
999995    1578.68
999996     972.78
999997    1768.00
999998    1695.68
999999    1198.07
Name: Premium, Length: 1000000, dtype: float64